In [1]:
import numpy as np
import requests
import pandas as pd
import time
import random
import statsmodels.api as sm
import  ta

from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
from futures_sign import send_signed_request, send_public_request
from cred import KEY, SECRET

In [2]:
client = Client(KEY, SECRET)
SYMBOL = 'ETHUSDT'
LIMIT = 1000
INTERVAL = '1m'
LEVERAGE = 10
TOTAL_CAF = 0.1
stop_percent = 0.005

fees = client.get_trade_fee(symbol=SYMBOL)
maker_fee = float(fees[0]['makerCommission'])
taker_fee = float(fees[0]['takerCommission'])

counterr = 0
POINTER = str(random.randint(1000, 9999))
URL = f'https://binance.com/fapi/v1/klines?symbol={SYMBOL}&limit={LIMIT}&interval={INTERVAL}'

In [3]:
def get_random_user_agent():
    os_version = f"Windows NT 10.0; Win64; x64"
    webkit_version = f"AppleWebKit/{random.randint(500, 600)}.{random.randint(10, 99)}"
    chrome_version = f"Chrome/{random.randint(100, 150)}.{random.randint(0, 9)}.{random.randint(0, 9)}.{random.randint(0, 9)}"
    safari_version = f"Safari/{random.randint(500, 600)}.{random.randint(10, 99)}"
    user_agent = f"Mozilla/5.0 ({os_version}) {webkit_version} (KHTML, like Gecko) {chrome_version} {safari_version}"
    return user_agent
print(get_random_user_agent())

Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/503.92 (KHTML, like Gecko) Chrome/133.1.7.8 Safari/549.55


In [4]:
def get_futures_klines(SYMBOL, INTERVAL, LIMIT):
    user_agent = get_random_user_agent()
    headers = {
        "User-Agent": user_agent,
    }
    endpoint = f'https://fapi.binance.com/fapi/v1/klines'
    params = {
        "symbol": SYMBOL,
        "interval": INTERVAL,
        "limit": LIMIT
    }

    try:
        response = requests.get(url=endpoint, headers=headers, params=params)
        response.raise_for_status()  # Check the response status
        data = response.json()
    except requests.RequestException as e:
        print(f"Error getting klines data: {e}")
        return None

    df = pd.DataFrame(data, columns=['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
    df = df.drop(['quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'], axis=1)
    df = df.astype({"open": float, "high": float, "low": float, "close": float, "volume": float})
    return df
print(get_futures_klines(SYMBOL, INTERVAL, LIMIT))

         open_time     open     high      low    close    volume  \
0    1703975880000  2298.19  2298.19  2297.73  2297.73   145.491   
1    1703975940000  2297.74  2297.74  2296.94  2297.06   475.717   
2    1703976000000  2297.05  2297.06  2295.08  2296.23  1830.466   
3    1703976060000  2296.23  2297.10  2296.22  2297.09   594.943   
4    1703976120000  2297.10  2297.20  2297.09  2297.19    79.149   
..             ...      ...      ...      ...      ...       ...   
995  1704035580000  2306.60  2306.97  2306.60  2306.91   268.225   
996  1704035640000  2306.92  2307.68  2306.82  2307.68   584.832   
997  1704035700000  2307.68  2308.53  2307.45  2308.10  1066.143   
998  1704035760000  2308.11  2308.71  2308.10  2308.71   591.717   
999  1704035820000  2308.70  2309.58  2308.70  2309.48   737.397   

        close_time  
0    1703975939999  
1    1703975999999  
2    1703976059999  
3    1703976119999  
4    1703976179999  
..             ...  
995  1704035639999  
996  1704035699

In [5]:
def get_symbol_price(SYMBOL):
    headers = {
        "accept-encoding": "gzip, deflate, br",
        "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
        "user-agent": get_random_user_agent(),
    }
    INTERVAL = '1m'
    try:
        # Используем метод, соответствующий вашим требованиям
        url = f'https://fapi.binance.com/fapi/v1/klines?symbol={SYMBOL}&interval={INTERVAL}&limit=1'
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()

        # Извлекаем текущую цену из ответа
        current_price = float(data[0][4])
        return current_price
    except requests.RequestException as e:
        print(f"Error getting price for symbol {SYMBOL}: {e}")
        return None
# Пример использования:
SYMBOL = 'ETHUSDT'
print(get_symbol_price(SYMBOL))

2309.49


In [6]:
def get_max_position(client, SYMBOL, TOTAL_CAF, LEVERAGE):
    symbol_price = get_symbol_price(SYMBOL)
    wallet = client.futures_account()

    balance = wallet.get('totalWalletBalance')
    if balance is None:
        print("totalWalletBalance is not available in the wallet.")
        return None

    balance = round(float(balance), 3)

    print(f"balance: {balance}")
    print(f"TOTAL_CAF: {TOTAL_CAF}")
    print(f"symbol_price: {symbol_price}")

    if symbol_price is not None:
        maxposition = round((balance * TOTAL_CAF * LEVERAGE) / symbol_price, 3)
        return maxposition
    else:
        print(f"Unable to calculate max position for {SYMBOL}.")
        return None
print(get_max_position(client, SYMBOL, TOTAL_CAF, LEVERAGE))

balance: 21.122
TOTAL_CAF: 0.1
symbol_price: 2309.49
0.009


In [ ]:
entry_price =  2280.41
current_price =  2276.47
fees = client.get_trade_fee(symbol=SYMBOL)
maker_fee = float(fees[0]['makerCommission'])
taker_fee = float(fees[0]['takerCommission'])
LEVERAGE = 20
maxposition = 0.03
opening_commission = entry_price * maxposition * float(maker_fee) / 2
closing_commission = current_price * maxposition * float(taker_fee) / 2
print('opening_commission:', opening_commission)
print('closing_commission:', closing_commission)
# take_profit = (current_price-entry_price) * maxposition - (opening_commission + closing_commission) # для лонга
take_profit = (entry_price - current_price) * maxposition - (opening_commission + closing_commission) #  для шорта
a = maxposition / LEVERAGE * entry_price
target_profit = (take_profit / a * 100)
c = (1- (current_price / entry_price)) * 100
print('take_profit в $=', take_profit)
print('target_profit % =', target_profit)
print("изменение цены на %:", c)
# fees = client.get_trade_fee(symbol=SYMBOL)
# print(fees)

In [ ]:
def open_position(SYMBOL, s_l, maxposition, INTERVAL, close_position=False):
    sprice = get_symbol_price(SYMBOL, INTERVAL)
    stop_price = 0

    if s_l == 'long':
        stop_price = str(round(sprice * (1 + stop_percent), 4))
    elif s_l == 'short':
        stop_price = str(round(sprice * (1 - stop_percent), 4))

    if stop_price:
        params = {
            "batchOrders": [
                {
                    "symbol": SYMBOL,
                    "side": "BUY" if s_l == 'long' else "SELL",
                    "type": "LIMIT",
                    "quantity": str(maxposition),
                    "timeInForce": "GTC",
                    "price": stop_price,
                    "closePosition": close_position
                    # Set closePosition to True if you want to close the position when the order is triggered
                }
            ]
        }

        response = send_signed_request('POST', '/fapi/v1/order', params)
        print(response)

In [ ]:
def open_position(SYMBOL, s_l, maxposition, INTERVAL, close_position=False):
    sprice = get_symbol_price(SYMBOL, INTERVAL)
    stop_price = 0

    if s_l == 'long':
        stop_price = str(round(sprice * (1 + stop_percent), 4))
    elif s_l == 'short':
        stop_price = str(round(sprice * (1 - stop_percent), 4))

    if stop_price:
        params = {
            "batchOrders": [
                {
                    "symbol": SYMBOL,
                    "side": "BUY" if s_l == 'long' else "SELL",
                    "type": "LIMIT",
                    "quantity": str(maxposition),
                    "timeInForce": "GTC",
                    "price": stop_price,
                    "closePosition": close_position
                }
            ]
        }

        response = send_signed_request('POST', '/fapi/v1/order', params)

        if "orderId" in response:
            print(f"Order opened successfully. Order ID: {response['orderId']}")
        else:
            print(f"Failed to open order. Error message: {response['msg']}")

# Пример вызова функции
SYMBOL = "BTCUSDT"
s_l = "long"
maxposition = 0.012
close_position = False
open_position(SYMBOL, s_l, maxposition, close_position)

In [ ]:
def close_position(SYMBOL, s_l, quantity_l,INTERVAL):
    sprice = get_symbol_price(SYMBOL,INTERVAL)
    stop_price = 0

    if s_l == 'long':
        stop_price = str(round(sprice * (1 - stop_percent), 4))
    elif s_l == 'short':
        stop_price = str(round(sprice * (1 + stop_percent), 4))

    if stop_price:
        params = {
            "symbol": SYMBOL,
            "side": "SELL" if s_l == 'long' else "BUY",
            "type": "LIMIT",
            "quantity": str(quantity_l),
            "timeInForce": "GTC",
            "price": stop_price
        }

        response = send_signed_request('POST', '/fapi/v1/order', params)
        print(response)
SYMBOL = "ETHUSDT"
s_l = "long"
quantity_l = 0.02
# should_close_position = False
close_position(SYMBOL, s_l, quantity_l, INTERVAL)

In [ ]:
def close_position(SYMBOL, s_l, quantity_l):
    params = {
        "symbol": SYMBOL,
        "side": "SELL" if s_l == 'long' else "BUY",
        "type": "MARKET",  # Изменяем тип ордера на MARKET
        "quantity": str(quantity_l),
        "timeInForce": "GTC"
    }

    response = send_signed_request('POST', '/fapi/v1/order', params)
    print(response)

In [ ]:
def get_opened_positions(client, SYMBOL):
    try:
        status = client.futures_account()
        positions = pd.DataFrame(status.get('positions', []))
        position_data = positions[positions['symbol'] == SYMBOL]

        if position_data.empty:
            return ["", 0, 0, 0, 0, 0, 0]

        position_amt = position_data['positionAmt'].astype(float).iloc[0]
        leverage = int(position_data['leverage'].iloc[0])
        entryprice = position_data['entryPrice'].iloc[0]
        profit = float(status.get('totalUnrealizedProfit', 0))
        balance = round(float(status.get('totalWalletBalance', 0)), 2)

        pos = ""
        if position_amt > 0:
            pos = "long"
        elif position_amt < 0:
            pos = "short"

        return [pos, position_amt, profit, leverage, balance, round(float(entryprice), 3), 0]

    except Exception as e:
        print(f"Error getting opened positions: {e}")
        return ["", 0, 0, 0, 0, 0, 0]

position = get_opened_positions(client, SYMBOL)
print(position)

In [ ]:
def check_and_close_orders(client, SYMBOL):  #проверяет наличие открытых ордеров для указанного символа на бирже
    try:
        open_orders = client.futures_get_open_orders(symbol=SYMBOL)

        if open_orders:
            client.futures_cancel_all_open_orders(symbol=SYMBOL)
            return True  # Вернем True, если были открытые ордера и они были закрыты
    except Exception as e:
        print(f"Error checking and closing orders for {SYMBOL}: {e}")

    return False  # Вернем False, если не было открытых ордеров или их не удалось закрыть

check_and_close_orders(client, SYMBOL)

In [ ]:
def indSlope(series, n):
    array_sl = [0 for _ in range(n - 1)]
    for j in range(n, len(series) + 1):
        y = series[j - n:j]
        x = np.array(range(n))
        x_sc = (x - x.min()) / (x.max() - x.min())
        y_sc = (y - y.min()) / (y.max() - y.min())
        x_sc = sm.add_constant(x_sc)
        model = sm.OLS(y_sc, x_sc)
        results = model.fit()
        array_sl.append(results.params.iloc[-1])
    slope_angle = (np.rad2deg(np.arctan(np.array(array_sl))))
    return np.array(slope_angle)

In [ ]:
def indATR(source_DF, n):
    df = source_DF.copy()
    df['H-L'] = abs(df['high'] - df['low'])
    df['H-PC'] = abs(df['high'] - df['close'].shift(1))
    df['L-PC'] = abs(df['low'] - df['close'].shift(1))
    df['TR'] = df[['H-L', 'H-PC', 'L-PC']].max(axis=1, skipna=False)
    df['ATR'] = df['TR'].rolling(n).mean()
    df.drop(['H-L', 'H-PC', 'L-PC'], axis=1, inplace=True)
    return df

In [ ]:
def isLCC(DF, i):
    df = DF.copy()
    LCC = 0

    if df['close'][i] <= df['close'][i + 1] and df['close'][i] <= df['close'][i - 1] and df['close'][i + 1] > \
            df['close'][i - 1]:
        # найдено Дно
        LCC = i - 1;
    return LCC

def isHCC(DF, i):
    df = DF.copy()
    HCC = 0
    if df['close'][i] >= df['close'][i + 1] and df['close'][i] >= df['close'][i - 1] and df['close'][i + 1] < \
            df['close'][i - 1]:
        # найдена вершина
        HCC = i;
    return HCC

In [ ]:
def PrepareDF(DF, n=12, atr_period=14, slope_period=5):
    ohlc = DF.iloc[:, [0, 1, 2, 3, 4, 5]]
    ohlc.columns = ["date", "open", "high", "low", "close", "volume"]
    ohlc = ohlc.set_index('date')

    # Вычисляем максимум и минимум канала
    maxx, minn = getMaxMinChannel(ohlc, n)

    df = indATR(ohlc, atr_period).reset_index()
    df['slope'] = indSlope(df['close'], slope_period)
    df['channel_max'] = maxx  # Максимум канала
    df['channel_min'] = minn  # Минимум канала
    df['position_in_channel'] = (df['close'] - minn) / (maxx - minn)
    df = df.set_index('date')
    df = df.reset_index()
    return df

def getMaxMinChannel(DF, n, maxx=0, minn=None):
    if minn is None:
        minn = DF['low'].max()
    for i in range(1, min(n, len(DF))):
        if maxx < DF['high'].values[-i]:
            maxx = DF['high'].values[-i]
        if minn > DF['low'].values[-i]:
            minn = DF['low'].values[-i]
    return maxx, minn

In [ ]:
df = get_futures_klines(SYMBOL, INTERVAL, LIMIT)
df = df[::-1]
prepared_df = PrepareDF(df)
print(prepared_df)

lend = len(prepared_df)

prepared_df['hcc'] = [None] * lend
prepared_df['lcc'] = [None] * lend

# prepared_df['deal_o']=[None]*lend
# prepared_df['deal_c']=[None]*lend
# prepared_df['earn']=[None]*lend

# for i in range(4, lend - 1):
#     if isHCC(prepared_df, i) > 0:
#         prepared_df.at[i, 'hcc'] = prepared_df['close'][i]
#     if isLCC(prepared_df, i) > 0:
#         prepared_df.at[i, 'lcc'] = prepared_df['close'][i]
#     prepared_df.to_csv('data.csv', index=False)

In [ ]:
# Загрузите 'data.csv'
df = pd.read_csv('data.csv')

# Преобразуйте столбец 'date' в формат datetime
df['date'] = pd.to_datetime(df['date'], unit='ms')
df['date'] = df['date'].dt.strftime('%H:%M')  # Преобразование времени в формат ЧЧ:ММ

df['lcc'].fillna('--------', inplace=True)
df['hcc'].fillna('--------', inplace=True)
df = df[(df['lcc'] != '--------') | (df['hcc'] != '--------')]
# Создайте новый датафрейм с нужными столбцами

new_df = df[['date', 'lcc', 'hcc', 'slope', 'position_in_channel']]
new_df.to_csv('new_df.csv', index=False)

In [7]:
import pandas as pd
# Создайте список файлов, чтобы не повторять код
file_list = ["all_a.txt",
             "all_bb.txt",
             "all_pos.txt",
             "all_price.txt"]

# Инициализируйте пустой DataFrame для объединения данных
merged_df = pd.DataFrame()
# Читайте данные из файлов и объединяйте их
for file_name in file_list:
    try:
        # Используйте метод join для объединения по времени
        df = pd.read_csv(file_name, names=[ file_name.replace(".txt", "")], delimiter=" ")
        # df = df.set_index("time")
        if merged_df.empty:
            merged_df = df
        else:
            merged_df = merged_df.join(df)
    except FileNotFoundError:
        print(f"Warning: {file_name} not found. Skipping.")
# Сброс индекса, чтобы вернуть "time" в столбец
merged_df = merged_df.reset_index()
# Сохранение объединенной таблицы в Excel
merged_df.to_excel("merged_all.xlsx", index=False)

In [8]:
import pandas as pd
# Создайте список файлов, чтобы не повторять код
file_list = ["angle_slope.txt",
             "angle_positions.txt",
             "angle_price.txt",
             "angle_bb_percent.txt"]

# Инициализируйте пустой DataFrame для объединения данных
merged_df = pd.DataFrame()
# Читайте данные из файлов и объединяйте их
for file_name in file_list:
    try:
        # Используйте метод join для объединения по времени
        df = pd.read_csv(file_name, names=[ file_name.replace(".txt", "")], delimiter=" ")
        # df = df.set_index("time")
        if merged_df.empty:
            merged_df = df
        else:
            merged_df = merged_df.join(df)
    except FileNotFoundError:
        print(f"Warning: {file_name} not found. Skipping.")
# Сброс индекса, чтобы вернуть "time" в столбец
merged_df = merged_df.reset_index()
# Сохранение объединенной таблицы в Excel
merged_df.to_excel("merged_data.xlsx", index=False)

In [9]:
import pandas as pd
# Создайте список файлов, чтобы не повторять код
file_list = ['price.txt']
# Инициализируйте пустой DataFrame для объединения данных
merged_df = pd.DataFrame()
# Читайте данные из файлов и объединяйте их
for file_name in file_list:
    try:
        # Используйте метод join для объединения по времени
        df = pd.read_csv(file_name, names=[ file_name.replace(".txt", "")], delimiter=" ")
        # df = df.set_index("time")
        if merged_df.empty:
            merged_df = df
        else:
            merged_df = merged_df.join(df)
    except FileNotFoundError:
        print(f"Warning: {file_name} not found. Skipping.")
# Сброс индекса, чтобы вернуть "time" в столбец
merged_df = merged_df.reset_index()
# Сохранение объединенной таблицы в Excel
merged_df.to_excel("merged_price.xlsx", index=False)


In [10]:
import pandas as pd
# Создайте список файлов, чтобы не повторять код
file_list = ['slope.txt']
# Инициализируйте пустой DataFrame для объединения данных
merged_df = pd.DataFrame()
# Читайте данные из файлов и объединяйте их
for file_name in file_list:
    try:
        # Используйте метод join для объединения по времени
        df = pd.read_csv(file_name, names=[ file_name.replace(".txt", "")], delimiter=" ")
        # df = df.set_index("time")
        if merged_df.empty:
            merged_df = df
        else:
            merged_df = merged_df.join(df)
    except FileNotFoundError:
        print(f"Warning: {file_name} not found. Skipping.")

# Сброс индекса, чтобы вернуть "time" в столбец
merged_df = merged_df.reset_index()

# Сохранение объединенной таблицы в Excel
merged_df.to_excel("merged_slope.xlsx", index=False)

In [11]:
import pandas as pd
# Создайте список файлов, чтобы не повторять код
file_list = ['current_bb_percent.txt']
# Инициализируйте пустой DataFrame для объединения данных
merged_df = pd.DataFrame()
# Читайте данные из файлов и объединяйте их
for file_name in file_list:
    try:
        # Используйте метод join для объединения по времени
        df = pd.read_csv(file_name, names=[ file_name.replace(".txt", "")], delimiter=" ")
        # df = df.set_index("time")
        if merged_df.empty:
            merged_df = df
        else:
            merged_df = merged_df.join(df)
    except FileNotFoundError:
        print(f"Warning: {file_name} not found. Skipping.")
# Сброс индекса, чтобы вернуть "time" в столбец
merged_df = merged_df.reset_index()
# Сохранение объединенной таблицы в Excel
merged_df.to_excel("merged_bb_percent.xlsx", index=False)

In [12]:
import pandas as pd
# Создайте список файлов, чтобы не повторять код
file_list = ['position_in_channel.txt']
# Инициализируйте пустой DataFrame для объединения данных
merged_df = pd.DataFrame()
# Читайте данные из файлов и объединяйте их
for file_name in file_list:
    try:
        # Используйте метод join для объединения по времени
        df = pd.read_csv(file_name, names=[ file_name.replace(".txt", "")], delimiter=" ")
        # df = df.set_index("time")
        if merged_df.empty:
            merged_df = df
        else:
            merged_df = merged_df.join(df)
    except FileNotFoundError:
        print(f"Warning: {file_name} not found. Skipping.")
# Сброс индекса, чтобы вернуть "time" в столбец
merged_df = merged_df.reset_index()
# Сохранение объединенной таблицы в Excel
merged_df.to_excel("merged_position_in_channel.xlsx", index=False)

In [13]:
import pandas as pd
# Создайте список файлов, чтобы не повторять код
file_list = ['a_pos23.txt']
# Инициализируйте пустой DataFrame для объединения данных
merged_df = pd.DataFrame()
# Читайте данные из файлов и объединяйте их
for file_name in file_list:
    try:
        # Используйте метод join для объединения по времени
        df = pd.read_csv(file_name, names=[ file_name.replace(".txt", "")], delimiter=" ")
        # df = df.set_index("time")
        if merged_df.empty:
            merged_df = df
        else:
            merged_df = merged_df.join(df)
    except FileNotFoundError:
        print(f"Warning: {file_name} not found. Skipping.")
# Сброс индекса, чтобы вернуть "time" в столбец
merged_df = merged_df.reset_index()

# Сохранение объединенной таблицы в Excel
merged_df.to_excel("merged_a_pos23.xlsx", index=False)

In [14]:
import pandas as pd
# Создайте список файлов, чтобы не повторять код
file_list = ['a_price_23.txt']
# Инициализируйте пустой DataFrame для объединения данных
merged_df = pd.DataFrame()
# Читайте данные из файлов и объединяйте их
for file_name in file_list:
    try:
        # Используйте метод join для объединения по времени
        df = pd.read_csv(file_name, names=[ file_name.replace(".txt", "")], delimiter=" ")
        # df = df.set_index("time")
        if merged_df.empty:
            merged_df = df
        else:
            merged_df = merged_df.join(df)
    except FileNotFoundError:
        print(f"Warning: {file_name} not found. Skipping.")
# Сброс индекса, чтобы вернуть "time" в столбец
merged_df = merged_df.reset_index()
# Сохранение объединенной таблицы в Excel
merged_df.to_excel("merged_a_price_23.xlsx", index=False)

In [15]:
import pandas as pd
# Создайте список файлов, чтобы не повторять код
file_list = ['a_slope23.txt']
# Инициализируйте пустой DataFrame для объединения данных
merged_df = pd.DataFrame()
# Читайте данные из файлов и объединяйте их
for file_name in file_list:
    try:
        # Используйте метод join для объединения по времени
        df = pd.read_csv(file_name, names=[ file_name.replace(".txt", "")], delimiter=" ")
        # df = df.set_index("time")
        if merged_df.empty:
            merged_df = df
        else:
            merged_df = merged_df.join(df)
    except FileNotFoundError:
        print(f"Warning: {file_name} not found. Skipping.")
# Сброс индекса, чтобы вернуть "time" в столбец
merged_df = merged_df.reset_index()
# Сохранение объединенной таблицы в Excel
merged_df.to_excel("merged_a_slope23.xlsx", index=False)

In [16]:
import pandas as pd
# Создайте список файлов, чтобы не повторять код
file_list = ['angle_bb_23.txt']
# Инициализируйте пустой DataFrame для объединения данных
merged_df = pd.DataFrame()
# Читайте данные из файлов и объединяйте их
for file_name in file_list:
    try:
        # Используйте метод join для объединения по времени
        df = pd.read_csv(file_name, names=[ file_name.replace(".txt", "")], delimiter=" ")
        # df = df.set_index("time")
        if merged_df.empty:
            merged_df = df
        else:
            merged_df = merged_df.join(df)
    except FileNotFoundError:
        print(f"Warning: {file_name} not found. Skipping.")
# Сброс индекса, чтобы вернуть "time" в столбец
merged_df = merged_df.reset_index()
# Сохранение объединенной таблицы в Excel
merged_df.to_excel("merged_a_bb_23.xlsx", index=False)

In [17]:
import pandas as pd
# Создайте список файлов, чтобы не повторять код
file_list = ['a_vol17.txt']
# Инициализируйте пустой DataFrame для объединения данных
merged_df = pd.DataFrame()
# Читайте данные из файлов и объединяйте их
for file_name in file_list:
    try:
        # Используйте метод join для объединения по времени
        df = pd.read_csv(file_name, names=[ file_name.replace(".txt", "")], delimiter=" ")
        # df = df.set_index("time")
        if merged_df.empty:
            merged_df = df
        else:
            merged_df = merged_df.join(df)
    except FileNotFoundError:
        print(f"Warning: {file_name} not found. Skipping.")
# Сброс индекса, чтобы вернуть "time" в столбец
merged_df = merged_df.reset_index()
# Сохранение объединенной таблицы в Excel
merged_df.to_excel("merged_a_vol17.xlsx", index=False)

In [ ]:
import pandas as pd
current_bb_df = pd.read_csv("volume.txt", names=["time", "volume"], delimiter=" ")
current_bb_df = current_bb_df.set_index("time")
merged_df = pd.concat([current_bb_df], axis=1)
merged_df = merged_df.reset_index()
merged_df.to_excel("volume_bb.xlsx", index=False)


In [ ]:
import pandas as pd
import ta
import requests
client = Client(KEY, SECRET)
SYMBOL = 'ETHUSDT'
LIMIT = 150
INTERVAL = '1m'
LEVERAGE = 10
TOTAL_CAF = 0.1
stop_percent = 0.005


def get_current_bb_percent(KEY, SECRET, SYMBOL, INTERVAL, length=10, multiplier=2):
    # Получение исторических данных свечей
    api_url = f'https://api.binance.com/api/v1/klines'
    params = {'symbol': SYMBOL, 'interval': INTERVAL}
    response = requests.get(api_url, params=params)
    data = response.json()

    # Преобразование данных в DataFrame
    df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df['close'] = df['close'].astype(float)

    # Вычисление Bollinger Bands и процента Bollinger Bands
    df['bb_upper'], df['bb_middle'], df['bb_lower'] = ta.volatility.bollinger_hband(df['close'], window=length, std=multiplier)
    df['bb_percent'] = (df['close'] - df['bb_lower']) / (df['bb_upper'] - df['bb_lower']) * 100

    # Вывод текущего значения процента Bollinger Bands
    current_bb_percent = df['bb_percent'].iloc[-1]
    return current_bb_percent

# Замените YOUR_API_KEY и YOUR_API_SECRET на ваши ключи API Binance
api_key = 'YOUR_API_KEY'
api_secret = 'YOUR_API_SECRET'

# Пример использования функции
current_percent = get_current_bb_percent(KEY, SECRET, symbol='ETHUSDT', interval='1m', length=10, multiplier=2)
print(f'Текущее значение BB %: {current_percent}')


In [ ]:
import numpy as np

# Ваши данные
timestamps = [
    "21:08:42", "21:09:45", "21:10:48", "21:11:50", "21:12:53",
    "21:13:56", "21:14:59", "21:16:01", "21:17:04", "21:18:07",
    "21:19:09", "21:20:12"
]
values = [2061.64, 2060.71, 2061.14, 2059.9, 2060.99, 2059.78, 2060.1, 2060.76, 2060.41, 2062.06, 2060.84, 2060.8]

# Преобразование времени в числа (в секундах, например)
time_in_seconds = [i for i in range(len(timestamps))]

# Вычисление линейной регрессии (угла наклона) для последних 12 значений
slope, intercept = np.polyfit(time_in_seconds[-12:], values[-12:], 1)

# Определение направления движения
if slope > 0:
    direction = "положительный"
elif slope < 0:
    direction = "отрицательный"
else:
    direction = "ноль"

print(f"Направление движения: {direction}")
print(f"Угол наклона: {slope}")


In [ ]:
entry_prices = [2069.27, 2059.65, 2055, 2043.86, 2043.2, 2044.37, 2043.2, 2045.23, 2041.26, 2038.08, 2043.19, 2050.72, 2046.72]
current_prices = [2059.95, 2058.15, 2052.26, 2040.27, 2040.94, 2042.6, 2040.35, 2041.64, 2028.81, 2028.81, 2039.6, 2052.72, 2045.4]
fees = client.get_trade_fee(symbol=SYMBOL)
maker_fee = float(fees[0]['makerCommission'])
taker_fee = float(fees[0]['takerCommission'])
LEVERAGE = 10
maxposition = 0.015
max_profit = 0  # Initialize max_profit
for entry_price in entry_prices:
    opening_commission = entry_price * maxposition * maker_fee / 2
    for current_price in current_prices:
        closing_commission = current_price * maxposition * taker_fee / 2
        target_profit = (entry_price - current_price) * maxposition - (opening_commission + closing_commission)
        if target_profit >= max_profit:
            # Update max_profit if the target_profit is greater
            max_profit = target_profit
print('Max Profit:', max_profit)

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

def indSlope(series, n):
    array_sl = [0 for _ in range(n - 1)]
    for j in range(n, len(series) + 1):
        y = series[j - n:j]
        x = np.array(range(n))
        x_sc = (x - x.min()) / (x.max() - x.min())
        y_sc = (y - y.min()) / (y.max() - y.min())
        x_sc = sm.add_constant(x_sc)
        model = sm.OLS(y_sc, x_sc)
        results = model.fit()
        array_sl.append(results.params.iloc[-1])
    slope_angle = (np.rad2deg(np.arctan(np.array(array_sl))))
    return np.array(slope_angle)

def find_crossings(series1, series2):
    crossings = []

    for i in range(1, len(series1)):
        if (series1[i-1] < series2[i-1]) and (series1[i] > series2[i]):
            crossings.append(i)
        elif (series1[i-1] > series2[i-1]) and (series1[i] < series2[i]):
            crossings.append(i)

    return crossings

# Замените эти данные своими данными
data = {'Date': ['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05'],
        'MA7': [100, 105, 98, 110, 95],
        'MA13': [102, 100, 97, 105, 92]}

df = pd.DataFrame(data)
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# Вычисление угла наклона для MA7 и MA13 с использованием функции indSlope
df['MA7_slope'] = indSlope(df['MA7'], 7)
df['MA13_slope'] = indSlope(df['MA13'], 13)

# Нахождение точек пересечения
crossings = find_crossings(df['MA7'], df['MA13'])

# Вывод результатов
print("Точки пересечения MA7 и MA13:", crossings)


In [ ]:
### рисовалка
# import statsmodels.api as sm
import matplotlib.pyplot as plt

aa = prepared_df[0:100]
aa = aa.reset_index()

#labels = ['close',"deal_o","deal_c"]
labels = ['close', "deal_o", "deal_c", "channel_max", "channel_min"]

labels_line = ['--', "*-", "*-", "g-", "r-"]

j = 0
x = pd.DataFrame()
y = pd.DataFrame()
for i in labels:
    x[j] = aa['index']
    y[j] = aa[i]
    j = j + 1

fig, (ax1, ax2, ax3) = plt.subplots(3, 1)

fig.suptitle('Deals')
fig.set_size_inches(20, 10)

for j in range(0, len(labels)):
    ax1.plot(x[j], y[j], labels_line[j])

ax1.set_ylabel('Price')
ax1.grid(True)

ax2.plot(x[0], aa['earn'], 'g-')  #EMA
ax3.plot(x[0], aa['position_in_channel'], '.-')  #EMA

ax2.grid(True)
ax3.grid(True)

plt.show()

In [ ]:
def prt(message):
    # telegram message
    print(POINTER + ': ' + message)

In [ ]:
import datetime
import time

def main(SYMBOL):
    INTERVAL = '1m'  # Установите интервал на '1m'
    while True:
        signal = check_if_signal(SYMBOL)
        if signal:
            current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            current_price = get_symbol_price(SYMBOL)  # Получить текущую цену

            print(f"Signal received: {signal}")
            print(f"Time: {current_time}")
            print(f"Current Symbol Price: {current_price}")

            with open("signal.txt", "a") as file:
                file.write(f"{current_time} - Signal: {signal}, Symbol Price: {current_price}\n")

        else:
            print("No signal at the moment.")
        time.sleep(60)  # Подождать 1 минуту (60 секунд)

if __name__ == "__main__":
    main(SYMBOL)


In [ ]:
Ваша цель в данном коде - продолжить отслеживание цены и прибыли, пока целевая прибыль увеличивается, и закрыть позицию, если целевая прибыль уменьшается. Давайте улучшим ваш код:
Этот код продолжит отслеживать прибыль, даже если она уменьшается, и закроет позицию, если целевая прибыль продолжит падать.

def long_position_handler(SYMBOL, maxposition, INTERVAL, random_delay, client, position_closed):
    while True:
        current_price = get_symbol_price(SYMBOL, INTERVAL)
        position = get_opened_positions(client, SYMBOL)
        entry_price = position[5]  # enter price

        fees = client.get_trade_fee(symbol=SYMBOL)
        maker_fee = float(fees[0]['makerCommission'])
        taker_fee = float(fees[0]['takerCommission'])

        opening_commission = entry_price * position[1] * float(maker_fee) / 2
        closing_commission = current_price * position[1] * float(taker_fee) / 2

        take_profit = round((current_price - entry_price) * position[1] - (opening_commission + closing_commission), 5)
        max_profit = take_profit  # Инициализируем максимальную прибыль текущей целевой прибылью
        target_profit = round(position[2] - (opening_commission + closing_commission), 5)

        parser = argparse.ArgumentParser()
        parser.add_argument("--signal", help="Received signal from SEARCH_SIGNAL.py")
        args = parser.parse_args()

        if target_profit >= max_profit or args.signal == 'long':
            # Обновляем максимальную прибыль и продолжаем следить
            max_profit = target_profit
            time.sleep(random_delay)
        else:
            # Если целевая прибыль уменьшается, закрываем позицию и выводим прибыль
            if position[0] == 'long':
                close_position(SYMBOL, 'long', abs(position[1]))
                position_closed = True
                break  # Выход из цикла

            # Здесь вы можете добавить код для повторных попыток открытия сделки с использованием текущей цены
            new_current_price = get_symbol_price(SYMBOL, INTERVAL)
            new_target_profit = (entry_price - new_current_price) * position[1] - (opening_commission + closing_commission)

            if new_target_profit >= max_profit:
                # Если новая целевая прибыль снова увеличивается, продолжаем следить
                max_profit = new_target_profit
                time.sleep(random_delay)
            else:
                # Если новая целевая прибыль также уменьшается, закрываем позицию и выводим прибыль
                if position[0] == 'long':
                    close_position(SYMBOL, 'long', abs(position[1]))
                    position_closed = True
                    break  # Выход из цикла


In [ ]:
# def PrepareDF(DF, n=12):
#     ohlc = DF.iloc[:, [0, 1, 2, 3, 4, 5]]
#     ohlc.columns = ["date", "open", "high", "low", "close", "volume"]
#     ohlc = ohlc.set_index('date')
#
#     # Вычисляем максимум и минимум канала
#     maxx = 0
#     minn = ohlc['low'].max()
#     for i in range(1, n):
#         if maxx < ohlc['high'].iloc[-i]:
#             maxx = ohlc['high'].iloc[-i]
#         if minn > ohlc['low'].iloc[-i]:
#             minn = ohlc['low'].iloc[-i]
#
#     df = indATR(ohlc, 14).reset_index()
#     df['slope'] = indSlope(df['close'], 5)
#     df['channel_max'] = maxx  # Максимум канала
#     df['channel_min'] = minn  # Минимум канала
#     df['position_in_channel'] = (df['close'] - minn) / (maxx - minn)
#     df = df.set_index('date')
#     df = df.reset_index()
#     return df

In [ ]:
# def get_futures_klines(SYMBOL, LIMIT):
#     user_agent = get_random_user_agent()
#     headers = {
#         "accept-encoding": "gzip, deflate, br",
#         "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
#         "user-agent": user_agent,
#     }
#     try:
#         response = requests.get(url=URL, headers=headers)
#         response.raise_for_status()  # Check the response status
#         data = response.json()
#     except requests.RequestException as e:
#         print(f"Error getting klines data: {e}")
#         return None
#
#     df = pd.DataFrame(data)
#     df = pd.DataFrame(response.json())
#     df.columns = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'd1', 'd2', 'd3', 'd4', 'd5']
#     df = df.drop(['d1', 'd2', 'd3', 'd4', 'd5'], axis=1)
#     df['open'] = df['open'].astype(float)
#     df['high'] = df['high'].astype(float)
#     df['low'] = df['low'].astype(float)
#     df['close'] = df['close'].astype(float)
#     df['volume'] = df['volume'].astype(float)
#     return df
# print(get_futures_klines(SYMBOL, LIMIT))

In [ ]:
# последний норм рабочий 17112023

def check_if_signal(SYMBOL):
    ohlc = get_futures_klines(SYMBOL, INTERVAL, LIMIT)
    # current_price = get_symbol_price(SYMBOL)

    if ohlc is not None:
        prepared_df = PrepareDF(ohlc)
        # Debug print statement to check column names
        # print(prepared_df.columns)
        # Check column names
        if 'position_in_channel' not in prepared_df.columns or 'date' not in prepared_df.columns:
            print("Error: Incorrect column names in the DataFrame.")
            return None

        signals = []
        i = len(prepared_df) - 1
        # Check conditions for generating signals based on your specified logic

        # print(f"position_in_channel (i - 2): {prepared_df['position_in_channel'].iloc[i - 2]}")
        # print(f"position_in_channel (i - 1): {prepared_df['position_in_channel'].iloc[i - 1]}")
        # print(f"position_in_channel (i): {prepared_df['position_in_channel'].iloc[i]}")
        #
        # print(f"close (i - 2): {prepared_df['close'].iloc[i - 2]}")
        # print(f"close (i - 1): {prepared_df['close'].iloc[i - 1]}")
        # print(f"close (i): {prepared_df['close'].iloc[i]}")

        if (
            all(prepared_df['position_in_channel'].iloc[i - 2: i] > 0.74) and
            prepared_df['position_in_channel'].iloc[i - 1] > 0.95 and
            prepared_df['position_in_channel'].iloc[i - 1] > prepared_df['position_in_channel'].iloc[i] and
            prepared_df['close'].iloc[i - 2] <= prepared_df['close'].iloc[i - 1] and
            prepared_df['close'].iloc[i - 1] >= prepared_df['close'].iloc[i]
        ):
            signals.append(('short', prepared_df['date'].iloc[i]))

        elif (
            all(prepared_df['position_in_channel'].iloc[i - 2: i] < 0.26) and
            prepared_df['position_in_channel'].iloc[i - 1] < 0.05 and
            prepared_df['position_in_channel'].iloc[i - 1] > prepared_df['position_in_channel'].iloc[i] and
            prepared_df['close'].iloc[i - 2] >= prepared_df['close'].iloc[i - 1] and
            prepared_df['close'].iloc[i - 1] <= prepared_df['close'].iloc[i]
        ):
            signals.append(('long', prepared_df['date'].iloc[i]))

        return signals
    else:
        print("Error getting klines data. Check your internet connection.")
        return None

In [ ]:
# 21112023

        if (
                abs(last_slope[-1] - last_slope[-2]) > 33 and
                last_positions[-2] > 0.91 and
                last_positions[-1] <= last_positions[-2] and
                last_prices[-2] >= current_price
        ):
            signals.append(('short', current_time))

        if (
                abs(last_slope[-1] - last_slope[-2]) > 33 and
                last_positions[-1] < 0.09 and
                last_positions[-1] <= last_positions[-2] and
                last_prices[-2] >= current_price
        ):
            signals.append(('long', current_time))

In [ ]:
def check_if_signal(SYMBOL, current_time, current_price):
    ohlc = get_futures_klines(SYMBOL, INTERVAL, LIMIT)

    if ohlc is not None:
        prepared_df = PrepareDF(ohlc)

        if 'position_in_channel' not in prepared_df.columns or 'date' not in prepared_df.columns:
            print("Error: Incorrect column names in the DataFrame.")
            return None

        signals = []
        i = len(prepared_df) - 1

        try:
            with open('slope.txt', 'r') as f:
                lines = f.readlines()
                if len(lines) >= 7:
                    last_slope = [float(line.split()[1]) for line in lines [-7:]]
                    start_index_positions = len(lines) - 7
                    end_index_positions = len(lines)
                    angle_slope = np.polyfit(range(len(last_slope)), last_slope, 1)[0]

                    with open('angle_slope.txt', 'a+') as price_file:
                        current_time = time.strftime('%H:%M:%S')
                        current_entry = f"{current_time} {angle_slope}\n"
                        # Записываем новую запись в конец файла
                        price_file.write(current_entry)
                        # Сбрасываем указатель файла и читаем все записи
                        price_file.seek(0)
                        entries = price_file.readlines()
                        # Проверяем количество записей
                        if len(entries) >= 500:
                            # Если записей больше 500, обрезаем список до последних 500 записей
                            entries = entries[-500:]
                        # Сбрасываем указатель файла в начало и перезаписываем все записи
                        price_file.seek(0)
                        price_file.truncate()
                        # Записываем все записи обратно в файл
                        price_file.writelines(entries)

                        print('Angle slope:', angle_slope)
                        print("Last slope:", last_slope[-3:])
                        # print("Last slope:", last_slope if isinstance(last_slope, (list, np.ndarray)) else "Invalid data")

                else:
                    print("Error: Insufficient data in angle_slope.txt.")
                    return None
        except FileNotFoundError:
            print("Error: slope.txt not found.")
            return None

        try:
            with open('position_in_channel.txt', 'r') as f:
                lines = f.readlines()
                if len(lines) >= 7:
                    last_positions = [float(line.split()[1]) for line in lines[-7:]]
                    start_index_positions = len(lines) - 7
                    end_index_positions = len(lines)
                    angle_positions = np.polyfit(range(len(last_positions)), last_positions, 1)[0]

                    with open('angle_positions.txt', 'a+') as price_file:
                        current_time = time.strftime('%H:%M:%S')
                        current_entry = f"{current_time} {angle_positions}\n"
                        # Записываем новую запись в конец файла
                        price_file.write(current_entry)
                        # Сбрасываем указатель файла и читаем все записи
                        price_file.seek(0)
                        entries = price_file.readlines()
                        # Проверяем количество записей
                        if len(entries) >= 500:
                            # Если записей больше 100, обрезаем список до последних 100 записей
                            entries = entries[-500:]
                        # Сбрасываем указатель файла в начало и перезаписываем все записи
                        price_file.seek(0)
                        price_file.truncate()
                        # Записываем все записи обратно в файл
                        price_file.writelines(entries)

                    print('Angle positions:', angle_positions)
                    print("Last positions:", last_positions [-3:])
                    # print("Indexes:", list(range(start_index_positions, end_index_positions)))
                else:
                    print("Error: Insufficient data in position_in_channel.txt.")
                    return None
        except FileNotFoundError:
            print("Error: position_in_channel.txt not found.")
            return None

        try:
            with open('price.txt', 'r') as f:
                lines = f.readlines()
                if len(lines) >= 7:
                    last_prices = [float(line.split()[1]) for line in lines[-7:]]
                    start_index_prices = len(lines) - 7
                    end_index_prices = len(lines)
                    angle_prices = np.polyfit(range(len(last_prices)), last_prices, 1)[0]

                    print('Angle prices:', angle_prices)
                    print("Last prices:", last_prices [-3:])
                    # print("Indexes:", list(range(start_index_prices, end_index_prices)))
                else:
                    print("Error: Insufficient data in price.txt.")
                    return None
        except FileNotFoundError:
            print("Error: price.txt not found.")
            return None

        try:
            # Открываем файл current_bb_percent.txt для чтения
            with open('current_bb_percent.txt', 'r') as file:
                # Читаем все строки из файла
                lines = file.readlines()

                # Получаем последние пять записей
                last_five_entries = lines[-5:]

                # Проходимся по каждой записи
                for entry in last_five_entries:
                    # Разделяем запись на части (время и значение)
                    time, bb_percent = entry.split()

                    # Теперь у вас есть доступ к переменным time и bb_percent
                    # И вы можете использовать их в вашем коде

                    # Например:
                    print(f"Time: {time}, BB Percent: {bb_percent}")

        except FileNotFoundError:
            print("Error: current_bb_percent.txt not found.")
        except Exception as e:
            print(f"Error reading current_bb_percent.txt: {e}")

        if (
                last_slope[-2] > 46 and
                bb_percent > 110 and
                last_positions[-2] > 0.95  # and
        ):
            signals.append(('short', current_time))

        if (
                last_slope[-1] < - 40 and
                last_positions[-1] < 0.1 and
                last_positions[-1] < last_positions[-2]
        ):
            signals.append(('short_x', current_time))

        if (
                last_slope[-2] < - 44 and
                bb_percent < 0 and
                last_positions[-1] < 0.05
        ):
            signals.append(('long', current_time))

        if (
                last_slope[-1] > 40 and
                last_positions[-1] > 0.9 and
                last_positions[-1] < last_positions[-2]
        ):
            signals.append(('long_x', current_time))

        return signals
    else:
        print("Error getting klines data. Check your internet connection.")
        return None


In [ ]:
# 27112023  ////////////////////////////////////////////////////////////////////////////////////////
        if (
                (last_slope[-1] > 45 or last_slope[-2] > 45) and
                (last_positions[-1] > 0.8 or last_positions[-2] > 0.8) and
                last_positions[-2] > last_positions[-1] and
                bb_percent > 70
        ):
            signals.append(('short', current_time))

        if (
                (last_slope[-1] < -44 or last_slope[-2] < -44) and
                (last_positions[-1] < 0.05 or last_positions[-2] < 0.05) and
                last_positions[-1] > last_positions[-2] and
                bb_percent < 11
        ):
            signals.append(('short_x', current_time))

#////////////////////////////////////////////////////////////////////////////////////////

        if (
                (last_slope[-1] < -44 or last_slope[-2] < -44)  and
                (last_positions[-1] < 0.35 or last_positions[-2] < 0.35)  and
                last_positions[-2] > last_positions[-1] and
                bb_percent < 40
        ):
            signals.append(('long', current_time))

        if (
                (last_slope[-1] > 41 or last_slope[-2] > 41) and
                (last_positions[-1] > 0.95 or last_positions[-2] > 0.95) and
                last_positions[-1] > last_positions[-2] and
                bb_percent > 95
        ):
            signals.append(('long_x', current_time))
# ////////////////////////////////////////////////////////////////////////////////////////

In [ ]:
        # if (
        #         (last_slope[-1] < -35 or last_slope[-2] < -35) and
        #         (last_positions[-1] < 0.05 or last_positions[-1] < 0.05) and
        #         (bb_percent_1 < 15 or bb_percent_2 < 15)
        # ):
        #     signals.append(('1_2', current_time))

In [ ]:
        # if (
        #         (last_slope[-2] > 35 or last_slope[-1] < -5) and
        #         (last_positions[-1] < 0.3 or last_positions[-2] < 0.3) and
        #         (bb_percent_1 < 30 or bb_percent_2 < 30) and
        #         bb_percent_1 < bb_percent_2
        # ):
        #     signals.append(('3', current_time))

In [ ]:
        # if (
        #         (last_slope[-1] < -44 or last_slope[-2] < -44) and
        #         (last_positions[-1] < 0.15 or last_positions[-2] < 0.15) and
        #         last_positions[-2] > last_positions[-1] and
        #         bb_percent_1 < 20
        # ):
        #     signals.append(('long', current_time))

In [ ]:
            # if (
            #     (last_slope[-1] > 45 or last_slope[-2] > 45) and
            #     (last_positions[-1] > 0.8 or last_positions[-2] > 0.8) and
            #     last_positions[-2] > last_positions[-1] and
            #     bb_percent_1 > 80
            # ):
            #     signals.append(('short', current_time))

In [ ]:
        # if (
        #         (last_slope[-1] > 41 or last_slope[-2] > 41) and
        #         (last_positions[-1] > 0.95 or last_positions[-2] > 0.95) and
        #         last_positions[-1] > last_positions[-2] and
        #         (bb_percent_1 > 90 or bb_percent_1 > 90) and
        #         bb_percent_2 > bb_percent_1
        # ):
        #     signals.append(('6', current_time))

In [ ]:
        if (
                (last_slope[-2] < -43 or last_slope[-1] < -43) and
                (last_positions[-2] < 0.05 or last_positions[-1] < 0.05) and
                last_positions[-2] > last_positions[-1] and
                (bb_percent_2 < 10 or bb_percent_1 < 10) and
                bb_percent_2 > bb_percent_1 and
                angle_positions_1 < 0
        ):
            signals.append(('X_DOWN', current_time))

        if (
                # (volume_2 > volume_1) and
                (bb_percent_2 > 90 or bb_percent_2 > 90 or bb_percent_1 > 90) and
                bb_percent_2 > bb_percent_1 and
                (last_positions[-2] > 0.95 or last_positions[-1] > 0.95) and
                last_positions[-2] > last_positions[-1] and
                (last_slope[-3] > 45 or last_slope[-2] > 45 or last_slope[-1] > 45) and
                (angle_positions_2 > 0 or angle_positions_1 > 0) and
                (angle_slope_3 > 4 or angle_slope_2  > 4 or angle_slope_1  > 4) and
                (angle_slope_3 < angle_slope_2) and
                (angle_prices_2 > angle_prices_1)
        ):
            signals.append(('X_UP', current_time))

        if (
                # (volume_1 < 50) and
                (bb_percent_3 < 25  or bb_percent_2 < 25 or bb_percent_1 < 25) and
                (last_positions[-3] < 0.05 or last_positions[-2] < 0.05 or last_positions[-1] < 0.05) and
                (last_slope[-3] > -45 or last_slope[-2] < -45 or last_slope[-1] < -45) and
                (angle_positions_3 <0 and angle_positions_2 < 0 and angle_positions_1 > 0) and
                (angle_positions_2 < angle_positions_1) and
                (angle_slope_3 < -4 or angle_slope_2 < -4 or angle_slope_1 < -4) and
                (angle_slope_3 < angle_slope_1 ) and
                (angle_prices_2 < angle_prices_1)
        ):
            signals.append(('long_1', current_time))

        if (
                # (volume_2 < volume_1) and
                (bb_percent_3 > 0.85  or bb_percent_2 > 0.85 or bb_percent_1 > 0.85) and
                (last_positions[-3] > 0.9 or last_positions[-2] > 0.9 or last_positions[-1] < 0.9) and
                (last_slope[-3] > 42 or last_slope[-2] > 42 or last_slope[-1] > 42) and
                (last_slope[-3] > 0 and last_slope[-1] < 0) and
                (angle_positions_2 < angle_positions_1) and
                (angle_slope_3 > angle_slope_1 ) and
                (angle_prices_3 > angle_prices_2 > angle_prices_1)
        ):
            signals.append(('short_1', current_time))

        if (
                (volume_3 < 50 or volume_2 < 50 or volume_1 < 50)  and
                (bb_percent_3 < 20 or bb_percent_2 < 20 or bb_percent_1 < 20) and
                (last_positions [-3] < 0.05 or last_positions [-2] < 0.05 or last_positions [-1] < 0.05) and
                (last_slope[-3] < -45 or last_slope [-2] < -45 or last_slope[-1] < -45) and
                (angle_positions_3 < angle_positions_1) and
                (angle_slope_3 <-8 or angle_slope_2 <-8 or angle_slope_1 < -8) and
                (angle_prices_3 < -0.35 or  angle_prices_2 < -0.35 or angle_prices_1 < -0.35 )
        ):
            signals.append(('long_2', current_time))

        if (
                # (volume_1 < 50) and
                (bb_percent_3 > 85 or bb_percent_2 > 85 or bb_percent_1 > 85) and
                (last_positions [-3] > 0.85 or last_positions [-2] > 0.85 or last_positions [-1] > 0.85) and
                (last_slope[-3] > 45 or last_slope [-2] > 45 or last_slope[-1] > 45) and
                (angle_positions_3 < angle_positions_2 or angle_positions_2  < angle_positions_1) and
                (angle_slope_3 > 8 or angle_slope_2 > 8 or angle_slope_1 > 8 ) and
                (angle_slope_2 > angle_slope_1) and
                (angle_prices_3 < angle_prices_2 or angle_prices_2 < angle_prices_1)

        ):
            signals.append(('short_2', current_time))

In [ ]:
        if (
                any(angle_slope > 5 for angle_slope in [angle_slope_2, angle_slope_1]) and
                (
                any(angle_positions > 0.65 for angle_positions in [angle_positions_2, angle_positions_2, angle_positions_1]) or
                any(angle_prices > 0.54 for angle_prices in [angle_prices_3, angle_prices_2, angle_prices_1]) or
                any(angle_bb > 5 for angle_bb in [angle_bb_3, angle_bb_2, angle_bb_1])
                )
                and
                (
                any(bb_percent > 80 for bb_percent in [bb_percent_2, bb_percent_1]) or
                any(angle_positions > 0.9 for angle_positions in [angle_positions_2, angle_positions_1])
                )
                and all_angle > 10
                and (last_slope[-3] > 46 or last_slope[-2] > 46 or last_slope[-1] > 46)
        ):
            signals.append(('short_1', current_time))

        if (
                any(angle_slope < -5 for angle_slope in [angle_slope_3, angle_slope_2, angle_slope_1]) and
                (
                any(angle_positions < -0.05 for angle_positions in [angle_positions_3, angle_positions_2, angle_positions_1]) or
                any(angle_prices < -0.49 for angle_prices in [angle_prices_3, angle_prices_2, angle_prices_1]) or
                any(angle_bb < -5 for angle_bb in [angle_bb_3, angle_bb_2, angle_bb_1])
                )
                and
                (
                any(bb_percent < 20 for bb_percent in [bb_percent_3, bb_percent_2, bb_percent_1]) or
                any(angle_positions < 0.1 for angle_positions in [angle_positions_3, angle_positions_2, angle_positions_1])
                )
                and all_angle < -10
                and (last_slope[-3] < -46 or last_slope[-2] < -46 or last_slope[-1] < -46 )
        ):
            signals.append(('short_1_x', current_time))


In [ ]:
        if (
                ((last_slope[-3] > 45 or last_slope[-2] > 45) and last_slope[-1] > 45) and
                ((last_positions [-3] > 0.95 or last_positions [-2] > 0.95) and last_positions [-1] < 0.95) and
                ( last_vol3 < 500)
        ):
            signals.append(('UP', current_time))

        if (
                ((last_slope[-3] < -45 or last_slope[-2] < -45) and last_slope[-1] > -45) and
                ((last_positions[-3] < 0.05 or last_positions[-2] < 0.05) and last_positions[-1] > 0.05) and
                (last_vol3 > 2000)
        ):
            signals.append(('DOWN', current_time))

In [ ]:
        all_pos = round ((angle_positions + a_pos23)/2, 4)
        with open('all_pos.txt', 'a+') as file:
            current_entry = f"{current_time} {all_pos}\n"
            file.write(current_entry)
            file.seek(0)
            entries = file.readlines()
            if len(entries) >= 500:
                entries = entries[-500:]
            file.seek(0)
            file.truncate()
            file.writelines(entries)

            print('Angle all_pos:', all_pos)
        try:
            with open('all_pos.txt', 'r') as f:
                lines = f.readlines()
                # Проверка, что есть минимум две записи
                if len(lines) >= 3:
                    # Получаем последние две записи
                    entry_1 = lines[-1].split()
                    entry_2 = lines[-2].split()
                    entry_3 = lines[-3].split()
                    # Преобразование в числа
                    all_pos_1 = float(entry_1[1])
                    all_pos_2 = float(entry_2[1])
                    all_pos_3 = float(entry_3[1])
        except FileNotFoundError:
            print("Error: all_pos.txt not found.")
        except Exception as e:
            print(f"Error reading all_pos.txt: {e}")

        all_bb = round ((angle_bb_1 + a_bb_23) /2, 4)
        with open('all_bb.txt', 'a+') as file:
            current_entry = f"{current_time} {all_bb}\n"
            file.write(current_entry)
            file.seek(0)
            entries = file.readlines()
            if len(entries) >= 500:
                entries = entries[-500:]
            file.seek(0)
            file.truncate()
            file.writelines(entries)

            print('Angle all_bb:', all_bb)
        try:
            with open('all_bb.txt', 'r') as f:
                lines = f.readlines()
                # Проверка, что есть минимум две записи
                if len(lines) >= 3:
                    # Получаем последние две записи
                    entry_1 = lines[-1].split()
                    entry_2 = lines[-2].split()
                    entry_3 = lines[-3].split()
                    # Преобразование в числа
                    all_bb_1 = float(entry_1[1])
                    all_bb_2 = float(entry_2[1])
                    all_bb_3 = float(entry_3[1])
        except FileNotFoundError:
            print("Error: all_bb.txt not found.")
        except Exception as e:
            print(f"Error reading all_bb.txt: {e}")

        all_a = round ((angle_slope + all_angle23)/2 , 4)
        with open('all_a.txt', 'a+') as price_file:
            current_entry = f"{current_time} {all_a}\n"
            price_file.write(current_entry)
            price_file.seek(0)
            entries = price_file.readlines()
            if len(entries) >= 500:
                entries = entries[-500:]
            price_file.seek(0)
            price_file.truncate()
            price_file.writelines(entries)

            print('Angle all_a:', all_a)
        try:
            with open('all_a.txt', 'r') as f:
                lines = f.readlines()
                # Проверка, что есть минимум две записи
                if len(lines) >= 3:
                    # Получаем последние две записи
                    entry_1 = lines[-1].split()
                    entry_2 = lines[-2].split()
                    entry_3 = lines[-3].split()
                    # Преобразование в числа
                    all_a_1 = float(entry_1[1])
                    all_a_2 = float(entry_2[1])
                    all_a_3 = float(entry_3[1])
        except FileNotFoundError:
            print("Error: all_a.txt not found.")
        except Exception as e:
            print(f"Error reading all_a.txt: {e}")

        all_sum = round ((all_price + all_pos + all_bb + all_a), 4)
        with open('all_sum.txt', 'a+') as s_file:
            current_entry = f"{current_time} {all_sum}\n"
            s_file.write(current_entry)
            s_file.seek(0)
            entries = s_file.readlines()
            if len(entries) >= 500:
                entries = entries[-500:]
            s_file.seek(0)
            s_file.truncate()
            s_file.writelines(entries)

            print('Angle all_sum:', all_sum)
        try:
            with open('all_sum.txt', 'r') as f:
                lines = f.readlines()
                # Проверка, что есть минимум две записи
                if len(lines) >= 3:
                    # Получаем последние две записи
                    entry_1 = lines[-1].split()
                    entry_2 = lines[-2].split()
                    entry_3 = lines[-3].split()
                    # Преобразование в числа
                    all_sum_1 = float(entry_1[1])
                    all_sum_2 = float(entry_2[1])
                    all_sum_3 = float(entry_3[1])
        except FileNotFoundError:
            print("Error: all_sum.txt not found.")
        except Exception as e:
            print(f"Error reading all_sum.txt: {e}")

        all_price = round ((angle_prices_1 + a_pr23_1)/2, 4)
        with open('all_price.txt', 'a+') as file:
            current_entry = f"{current_time} {all_price}\n"
            file.write(current_entry)
            file.seek(0)
            entries = file.readlines()
            if len(entries) >= 500:
                entries = entries[-500:]
            file.seek(0)
            file.truncate()
            file.writelines(entries)

            print('Angle all_price:', all_price)
        try:
            with open('all_price.txt', 'r') as f:
                lines = f.readlines()
                # Проверка, что есть минимум две записи
                if len(lines) >= 3:
                    # Получаем последние две записи
                    entry_1 = lines[-1].split()
                    entry_2 = lines[-2].split()
                    entry_3 = lines[-3].split()
                    # Преобразование в числа
                    all_price_1 = float(entry_1[1])
                    all_price_2 = float(entry_2[1])
                    all_price_3 = float(entry_3[1])
        except FileNotFoundError:
            print("Error: all_price.txt not found.")
        except Exception as e:
            print(f"Error reading all_price.txt: {e}")

In [ ]:
def check_if_signal(SYMBOL, current_time, current_price):
    ohlc = get_futures_klines(SYMBOL, INTERVAL, LIMIT)

    if ohlc is not None:
        prepared_df = PrepareDF(ohlc)

        if 'position_in_channel' not in prepared_df.columns or 'date' not in prepared_df.columns:
            print("Error: Incorrect column names in the DataFrame.")
            return None

        signals = []
        i = len(prepared_df) - 1

        try:
            with open('slope.txt', 'r') as f:
                lines = f.readlines()
                if len(lines) >= 12:
                    last_slope = [float(line.split()[1]) for line in lines[-12:]]
                    start_index_positions = len(lines) - 12
                    end_index_positions = len(lines)
                    angle_slope = round (np.polyfit(range(len(last_slope)), last_slope, 1)[0], 4)

                    with open('angle_slope.txt', 'a+') as price_file:
                        current_entry = f"{current_time} {angle_slope}\n"
                        price_file.write(current_entry)
                        price_file.seek(0)
                        entries = price_file.readlines()
                        if len(entries) >= 500:
                            entries = entries[-500:]
                        price_file.seek(0)
                        price_file.truncate()
                        price_file.writelines(entries)

                        # print('Angle slope:', angle_slope)
                        # print("Last slope:", last_slope[-3:])

                if len(lines) >= 23:
                    last_slope = [float(line.split()[1]) for line in lines[-23:]]
                    start_index_positions = len(lines) - 23
                    end_index_positions = len(lines)
                    a_slope23 = round( np.polyfit(range(len(last_slope)), last_slope, 1)[0],4)

                    with open('a_slope23.txt', 'a+') as price_file:
                        current_entry = f"{current_time} {a_slope23}\n"
                        price_file.write(current_entry)
                        price_file.seek(0)
                        entries = price_file.readlines()
                        if len(entries) >= 500:
                            entries = entries[-500:]
                        price_file.seek(0)
                        price_file.truncate()
                        price_file.writelines(entries)

                        # print('Angle a_slope23:', a_slope23)

                else:
                    print("Error: Insufficient data in angle_slope.txt.")
                    return None
        except FileNotFoundError:
            print("Error: slope.txt not found.")
            return None
        try:
            with open('position_in_channel.txt', 'r') as f:
                lines = f.readlines()
                if len(lines) >= 12:
                    last_positions = [float(line.split()[1]) for line in lines[-12:]]
                    start_index_positions = len(lines) - 12
                    end_index_positions = len(lines)
                    angle_positions = round (np.polyfit(range(len(last_positions)), last_positions, 1)[0], 4)

                    with open('angle_positions.txt', 'a+') as posit_file:
                        current_entry = f"{current_time} {angle_positions}\n"
                        posit_file.write(current_entry)
                        posit_file.seek(0)
                        entries = posit_file.readlines()
                        if len(entries) >= 500:
                            entries = entries[-500:]
                        posit_file.seek(0)
                        posit_file.truncate()
                        posit_file.writelines(entries)

                    # print('Angle positions:', angle_positions)
                    # print("Last positions:", last_positions[-3:])
                if len(lines) >= 23:
                    last_positions = [float(line.split()[1]) for line in lines[-23:]]
                    start_index_positions = len(lines) - 23
                    end_index_positions = len(lines)
                    a_pos23 = round (np.polyfit(range(len(last_positions)), last_positions, 1)[0],4)

                    with open('a_pos23.txt', 'a+') as posit_file:
                        current_entry = f"{current_time} {a_pos23}\n"
                        posit_file.write(current_entry)
                        posit_file.seek(0)
                        entries = posit_file.readlines()
                        if len(entries) >= 500:
                            entries = entries[-500:]
                        posit_file.seek(0)
                        posit_file.truncate()
                        posit_file.writelines(entries)

                    # print('Angle a_pos23:', a_pos23)

                else:
                    print("Error: Insufficient data in position_in_channel.txt.")
                    return None
        except FileNotFoundError:
            print("Error: position_in_channel.txt not found.")
            return None
        try:
            with open('price.txt', 'r') as f:
                lines = f.readlines()
                if len(lines) >= 12:
                    last_prices = [float(line.split()[1]) for line in lines[-12:]]
                    start_index_prices = len(lines) -12
                    end_index_prices = len(lines)
                    angle_prices = round (np.polyfit(range(len(last_prices)), last_prices, 1)[0], 4)

                    with open('angle_price.txt', 'a+') as price_file:
                        current_entry = f"{current_time} {angle_prices}\n"
                        price_file.write(current_entry)
                        price_file.seek(0)
                        entries = price_file.readlines()
                        if len(entries) >= 500:
                            entries = entries[-500:]
                        price_file.seek(0)
                        price_file.truncate()
                        price_file.writelines(entries)

                    # print('Angle prices:', angle_prices)
                    # print("Last prices:", last_prices[-3:])

                    if len(lines) >= 23:
                        last_prices = [float(line.split()[1]) for line in lines[-23:]]
                        start_index_prices = len(lines) - 23
                        end_index_prices = len(lines)
                        a_prices23 = round (np.polyfit(range(len(last_prices)), last_prices, 1)[0],4)

                        with open('a_price_23.txt', 'a+') as price_file:
                            current_entry = f"{current_time} {a_prices23}\n"
                            price_file.write(current_entry)
                            price_file.seek(0)
                            entries = price_file.readlines()
                            if len(entries) >= 500:
                                entries = entries[-500:]
                            price_file.seek(0)
                            price_file.truncate()
                            price_file.writelines(entries)

                        # print('Angle prices 23:', a_prices23)

                else:
                    print("Error: Insufficient data in price.txt.")
                    return None
        except FileNotFoundError:
            print("Error: price.txt not found.")
            return None
        try:
            with open('current_bb_percent.txt', 'r') as f:
                lines = f.readlines()

                if len(lines) >= 12:
                    last_bb_percent = [float(line.split()[1]) for line in lines[-12:]]
                    start_index_prices = len(lines) -12
                    end_index_prices = len(lines)
                    angle_bb_percent = round (np.polyfit(range(len(last_bb_percent)), last_bb_percent, 1)[0],4)

                    with open('angle_bb_percent.txt', 'a+') as bb_file:
                        current_entry = f"{current_time} {angle_bb_percent}\n"
                        bb_file.write(current_entry)
                        bb_file.seek(0)
                        entries = bb_file.readlines()
                        if len(entries) >= 500:
                            entries = entries[-500:]
                        bb_file.seek(0)
                        bb_file.truncate()
                        bb_file.writelines(entries)

                    # print('angle_bb_percent prices:', angle_bb_percent)

                    if len(lines) >= 23:
                        last_bb_percent = [float(line.split()[1]) for line in lines[-23:]]
                        start_index_prices = len(lines) - 23
                        end_index_prices = len(lines)
                        a_bb_23 = round (np.polyfit(range(len(last_bb_percent)), last_bb_percent, 1)[0],4)

                        with open('angle_bb_23.txt', 'a+') as bb_file:
                            current_entry = f"{current_time} {a_bb_23}\n"
                            bb_file.write(current_entry)
                            bb_file.seek(0)
                            entries = bb_file.readlines()
                            if len(entries) >= 500:
                                entries = entries[-500:]
                            bb_file.seek(0)
                            bb_file.truncate()
                            bb_file.writelines(entries)

                        # print('angle_bb_23 prices:', a_bb_23)
                    # print("Last bb_percent:", last_bb_percent[-3:])

                else:
                    print("Error: Insufficient data in angle_bb_percent.txt.")
                    return None

                # Проверка, что есть минимум 3 записи
                if len(lines) >= 3:
                    # Получаем последние две записи
                    entry_1 = lines[-1].split()
                    entry_2 = lines[-2].split()
                    entry_3 = lines[-3].split()
                    # Преобразование в числа
                    bb_percent_1 = float(entry_1[1])
                    bb_percent_2 = float(entry_2[1])
                    bb_percent_3 = float(entry_3[1])
        except FileNotFoundError:
            print("Error: current_bb_percent.txt not found.")
        except Exception as e:
            print(f"Error reading current_bb_percent.txt: {e}")
        try:
            with open('volume.txt', 'r') as f:
                lines = f.readlines()
                # Проверка, что есть минимум две записи
                if len(lines) >= 3:
                    # Получаем последние две записи
                    entry_1 = lines[-1].split()
                    entry_2 = lines[-2].split()
                    entry_3 = lines[-3].split()
                    # Преобразование в числа
                    volume_1 = float(entry_1[1])
                    volume_2 = float(entry_2[1])
                    volume_3 = float(entry_3[1])
        except FileNotFoundError:
            print("Error: volume.txt not found.")
        except Exception as e:
            print(f"Error reading volume.txt: {e}")
        try:
            with open('angle_slope.txt', 'r') as f:
                lines = f.readlines()
                # Проверка, что есть минимум две записи
                if len(lines) >= 3:
                    # Получаем последние две записи
                    entry_1 = lines[-1].split()
                    entry_2 = lines[-2].split()
                    entry_3 = lines[-3].split()
                    # Преобразование в числа
                    angle_slope_1 = float(entry_1[1])
                    angle_slope_2 = float(entry_2[1])
                    angle_slope_3 = float(entry_3[1])
        except FileNotFoundError:
            print("Error: angle_slope.txt not found.")
        except Exception as e:
            print(f"Error reading angle_slope.txt: {e}")
        try:
            with open('angle_positions.txt', 'r') as f:
                lines = f.readlines()
                # Проверка, что есть минимум две записи
                if len(lines) >= 3:
                    # Получаем последние две записи
                    entry_1 = lines[-1].split()
                    entry_2 = lines[-2].split()
                    entry_3 = lines[-3].split()
                    # Преобразование в числа
                    angle_positions_1 = float(entry_1[1])
                    angle_positions_2 = float(entry_2[1])
                    angle_positions_3 = float(entry_3[1])
        except FileNotFoundError:
            print("Error: angle_positions.txt not found.")
        except Exception as e:
            print(f"Error reading angle_positions.txt: {e}")
        try:
            with open('angle_price.txt', 'r') as f:
                lines = f.readlines()
                # Проверка, что есть минимум две записи
                if len(lines) >= 3:
                    # Получаем последние две записи
                    entry_1 = lines[-1].split()
                    entry_2 = lines[-2].split()
                    entry_3 = lines[-3].split()
                    # Преобразование в числа
                    angle_prices_1 = float(entry_1[1])
                    angle_prices_2 = float(entry_2[1])
                    angle_prices_3 = float(entry_3[1])
        except FileNotFoundError:
            print("Error: angle_price.txt not found.")
        except Exception as e:
            print(f"Error reading angle_price.txt: {e}")
        try:
            with open('angle_bb_percent.txt', 'r') as f:
                lines = f.readlines()
                # Проверка, что есть минимум две записи
                if len(lines) >= 3:
                    # Получаем последние две записи
                    entry_1 = lines[-1].split()
                    entry_2 = lines[-2].split()
                    entry_3 = lines[-3].split()
                    # Преобразование в числа
                    angle_bb_1 = float(entry_1[1])
                    angle_bb_2 = float(entry_2[1])
                    angle_bb_3 = float(entry_3[1])
        except FileNotFoundError:
            print("Error: angle_bb_percent.txt not found.")
        except Exception as e:
            print(f"Error reading angle_bb_percent.txt: {e}")
        try:
            with open('a_price_23.txt', 'r') as f:
                lines = f.readlines()
                # Проверка, что есть минимум две записи
                if len(lines) >= 3:
                    # Получаем последние две записи
                    entry_1 = lines[-1].split()
                    entry_2 = lines[-2].split()
                    entry_3 = lines[-3].split()
                    # Преобразование в числа
                    a_pr23_1 = float(entry_1[1])
                    a_pr23_2 = float(entry_2[1])
                    a_pr23_3 = float(entry_3[1])
        except FileNotFoundError:
            print("Error: angle_price.txt not found.")
        except Exception as e:
            print(f"Error reading a_price_23.txt: {e}")
        try:
            with open('angle_bb_23.txt', 'r') as f:
                lines = f.readlines()
                # Проверка, что есть минимум две записи
                if len(lines) >= 3:
                    # Получаем последние две записи
                    entry_1 = lines[-1].split()
                    entry_2 = lines[-2].split()
                    entry_3 = lines[-3].split()
                    # Преобразование в числа
                    a_bb23_1 = float(entry_1[1])
                    a_bb23_2 = float(entry_2[1])
                    a_bb23_3 = float(entry_3[1])
        except FileNotFoundError:
            print("Error: angle_bb_23.txt not found.")
        except Exception as e:
            print(f"Error reading angle_bb_23.txt: {e}")
        try:
            with open('a_pos23.txt', 'r') as f:
                lines = f.readlines()
                # Проверка, что есть минимум две записи
                if len(lines) >= 3:
                    # Получаем последние две записи
                    entry_1 = lines[-1].split()
                    entry_2 = lines[-2].split()
                    entry_3 = lines[-3].split()
                    # Преобразование в числа
                    a_pos23_1 = float(entry_1[1])
                    a_pos23_2 = float(entry_2[1])
                    a_pos23_3 = float(entry_3[1])
        except FileNotFoundError:
            print("Error: angle_price.txt not found.")
        except Exception as e:
            print(f"Error reading a_pos23.txt: {e}")